In [12]:
import numpy as np

def seg_eval(pred, label, clss=[0, 1]):
    """
    calculate the dice between prediction and ground truth
    input:
        pred: predicted mask
        label: groud truth
        clss: eg. [0, 1] for binary class
    """
    Ncls = len(clss)
    dices = np.zeros(Ncls)
    [depth, height, width] = pred.shape
    for i in range(label.shape[-1]):
        if np.sum(label[:,:,i]) == 0:
            pred[:,:,i] = 0
    for idx, cls in enumerate(clss):
        # binary map
        pred_cls = np.zeros([depth, height, width])
        pred_cls[np.where(pred == cls)] = 1
        label_cls = np.zeros([depth, height, width])
        label_cls[np.where(label == cls)] = 1

        # cal the inter & conv
        s = pred_cls + label_cls
        inter = len(np.where(s >= 2)[0])
        conv = len(np.where(s >= 1)[0]) + inter
        try:
            dice = 2.0 * inter / conv
        except:
            print("conv is zeros when dice = 2.0 * inter / conv")
            dice = -1

        dices[idx] = dice

    return dices, pred

In [22]:
import os
import glob
pred = glob.glob(r'D:\20220421-FANBIN\pred/*.nii.gz')
gt = [os.path.join(r'D:/20220421-FANBIN/masks_12/', os.path.basename(p))for p in pred]
os.makedirs(r'D:/20220421-FANBIN/post_pred', exist_ok=True)

In [23]:
import nibabel as nib

dice = []
for p, g in zip(pred, gt):
    print(p, g)
    p_a = np.array(nib.load(p).dataobj)
    g_img = nib.load(g)
    g_a = np.array(g_img.dataobj)
    metric, post_pred = seg_eval(p_a, g_a, clss=[1,2])
    dice.append(metric)
    img = nib.Nifti1Image(post_pred, g_img.affine)
    nib.save(img, f"D:/20220421-FANBIN/post_pred/{os.path.basename(g)}")
    print(dice[-1])
print(np.mean(np.array(dice), axis=0))
print(np.mean(np.array(dice)[:,[True, True]]))

D:\20220421-FANBIN\pred\1708204504.nii.gz D:/20220421-FANBIN/masks_12/1708204504.nii.gz
[0.89878217 0.68985507]
D:\20220421-FANBIN\pred\17098412.nii.gz D:/20220421-FANBIN/masks_12/17098412.nii.gz
[0.85528814 0.765625  ]
D:\20220421-FANBIN\pred\1710035907.nii.gz D:/20220421-FANBIN/masks_12/1710035907.nii.gz
[0.77162414 0.67508205]
[0.84189815 0.71018737]
0.776042763169187


In [ ]:
D:\20220421-FANBIN\pred\1807270550.nii.gz D:/20220421-FANBIN/masks/1807270550.nii.gz
[0.75288531 0.69087523 0.74632035]
D:\20220421-FANBIN\pred\19043140.nii.gz D:/20220421-FANBIN/masks/19043140.nii.gz
[0.84062313 0.65756469 0.83737024]
D:\20220421-FANBIN\pred\1906088798.nii.gz D:/20220421-FANBIN/masks/1906088798.nii.gz
[0.79867841 0.46977673 0.73028238]

--校验前
[0.79739562 0.60607222 0.77132432]
0.784359969839653


D:\20220421-FANBIN\pred\1807270550.nii.gz D:/20220421-FANBIN/masks/1807270550.nii.gz
[0.91225827 0.76211997 0.75947137]
D:\20220421-FANBIN\pred\19043140.nii.gz D:/20220421-FANBIN/masks/19043140.nii.gz
[0.86272098 0.66420783 0.83737024]
D:\20220421-FANBIN\pred\1906088798.nii.gz D:/20220421-FANBIN/masks/1906088798.nii.gz
[0.87384022 0.50586396 0.73099415]

--校验后
[0.88293983 0.64406392 0.77594525]
0.8294425393681478